<a href="https://colab.research.google.com/github/dineshuthayakumar/AIExperiments/blob/main/Chatbot_openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modern AI Pro: Simple Chatbot on your enterprise content

We will scrape the web content as well as our PDFs to answer user queries more effectively.


In [1]:
!pip install --upgrade --quiet  sentence-transformers gradio bs4 pypdf2 chromadb rapidocr-onnxruntime langchain langchain-community langchainhub langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 14.2 MB/s eta 

## Step 1: Scraping our web content and using that as the foundation.


In [2]:
# Load, chunk and index the contents of a blog/website.
from langchain_community.document_loaders import WebBaseLoader
import bs4
loader = WebBaseLoader(web_path=("https://community.kissflow.com/t/q6h9qlf/creating-and-managing-subitems"))
docs = loader.load()
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [3]:
docs[0]

Document(metadata={'source': 'https://community.kissflow.com/t/q6h9qlf/creating-and-managing-subitems', 'title': 'Creating and managing subitems - Using boards - Kissflow Community', 'description': 'Subitems help you break down complex tasks into multiple small tasks and track them efficiently.', 'language': 'en'}, page_content='\n\n\nCreating and managing subitems - Using boards - Kissflow Community\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to main content\n\n\n\n\n\n\nCommunity\n\n\n\n\nDeveloper Documentation\nStatus\n\n\nHow to enable Javascript\n\n\n\n\n\n\n\n\n\n\nSign Up\n\n\nLogin\n\n\n\n\n\n\n\n\n\n\nWelcome to the Kissflow Community!\n\n\n\n\n\n\n\n\nSearch for answers here or ask a question\n\n\n\n\n\n\nProduct Documentation\n\n\nðŸŽ¬ Getting started\nðŸ“’ User guide\nðŸ”Ž Quick references\nðŸ’¼ Troubleshooting guide\n\n\n\nProduct Updates\n\n\nPre-release notes\nProduct announcements\nTips and tricks\n\n\nLatest Activity\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHome\n\n\nPr

In [4]:
# Split and manage the texts
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [5]:
# Store embeddings in a Vector DB
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2", model_kwargs={"device": "cpu"})
vectorstore_web = Chroma.from_documents(documents=splits, embedding=embeddings)
retriever_web = vectorstore_web.as_retriever()

<ipython-input-5-1d2d4e9d287b>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2", model_kwargs={"device": "cpu"})
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optio

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
from google.colab import userdata
import os

# If you haven't already installed "openai" in your Colab environment, make sure to:
# !pip install openai

# From LangChain, import the OpenAI LLM
from langchain.chat_models import ChatOpenAI

# Replace this with your actual environment variable (if stored in colab form fields, etc.)
os.environ["OPENAI_API_KEY"] = userdata.get("OpenAPIKey")

# Instantiate the OpenAI LLM (adjust model name if desired, e.g. "gpt-4" or "gpt-3.5-turbo")
llm = ChatOpenAI(model_name="gpt-4")

# Pull in the prompt from your prompt hub
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

# From your LangChain Core imports
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Create your pipeline using OpenAI as the LLM
rag_web = (
    {"context": retriever_web | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
rag_web.invoke("Can I add 200 sub items within a Kissflow Board item?")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


'No, you cannot add 200 sub items within a Kissflow Board item. The maximum number of subitems you can add inside an item is 50.'

In [ ]:
!pip install -q -U langchain-groq
from google.colab import userdata
import os
from langchain_groq import ChatGroq
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
llm = ChatGroq(model_name="llama3-70b-8192")

In [ ]:
# Retrieve and generate using the relevant snippets of the blog.
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [ ]:
# Building the RAG
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
rag_web = (
    {"context": retriever_web | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_web.invoke("What do the users say about Modern AI Pro?")

"Users say that Modern AI Pro's training bootcamp is informative, eye-opening, and provides invaluable knowledge and skills. They express gratitude and appreciation for the training, and are excited to apply what they learned in real-world applications. Overall, the feedback is positive and recommends the bootcamp."